<a href="https://colab.research.google.com/github/20KMJ/ESAA7/blob/main/%EB%AA%A8%EB%8D%B8%ED%9B%88%EB%A0%A8_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **| 모델 훈련 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch04 연습문제 1, 5, 9, 10
- 개념 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

### **1. 수백만 개의 특성을 가진 훈련 세트에서는 어떤 선형 회귀 알고리즘을 사용할 수 있을까요?**
___


경사 하강법 (배치 경사 하강법, 확률적 경사 하강법, 미니배치 경사 하강법)

### **2. 배치 경사 하강법을 사용하고 에포크마다 검증 오차를 그래프로 나타내봤습니다. 검증 오차가 일정하게 상승되고 있다면 어떤 일이 일어나고 있는 걸까요? 이 문제를 어떻게 해결할 수 있나요?**
___

학습률이 너무 크기 때문에 이리저리 널뛰면서 최적점에서 점점 멀어져 발산하고 있을 수 있다. 그리드 탐색을 통해해 적절한 학습률을 찾아야 한다.

### **3. 릿지 회귀를 사용했을 때 훈련 오차가 검증 오차가 거의 비슷하고 둘 다 높았습니다. 이 모델에는 높은 편향이 문제인가요, 아니면 높은 분산이 문제인가요? 규제 하이퍼파라미터 $\alpha$를 증가시켜야 할까요 아니면 줄여야 할까요?**
___

- 해당 경우는 **높은 편향**값으로 인해 알고리즘이 데이터의 특징과 결과물과의 적절한 관계를 놓치게 만드는 과소적합 문제가 발생한 것으로 보인다. 따라서 규제 하이퍼파라미터 α를 **줄여** 모델의 제약을 줄여줘야한다.

### **4. 다음과 같이 사용해야 하는 이유는?**
___
- 평범한 선형 회귀(즉, 아무런 규제가 없는 모델) 대신 릿지 회귀
- 릿지 회귀 대신 라쏘 회귀
- 라쏘 회귀 대신 엘라스틱넷

1. 적어도 규제가 약간 있는 것이 대부분의 경우에 좋으므로 일반적인 평범한 선형 회귀는 피해야 한다.
2. 릿지 회귀가 기본이 되지만 쓰이는 특성이 몇 개뿐이라고 의심되면 라쏘 회귀가 더 낫다.
3. 특성 수가 훈련 샘플 수보다 많거나 특성 몇 개가 강하게 연관되어 있을 때는 보통 라쏘 회귀가 문제를 일으키므로 엘라스틱넷이 더 낫다.

### **5. 조기 종료를 사용한 배치 경사 하강법으로 소프트맥스 회귀를 구현해보세요(사이킷런은 사용하지 마세요)**


---



In [ ]:
import numpy as np

from sklearn import datasets
iris = datasets.load_iris()
X = iris['data'][:,(2,3)]  # 꽃잎 길이, 꽃잎 넓이
y = iris['target']


# Add Bias
X_with_bias = np.c_[np.ones([len(X),1]),X]   
# len(X) 개수 만큼 1로 채워진 [ ], [ ]......[ ] array
# 2열 -> 3열로 늘어남


# 결과를 일정하게 하기 위해, random seed 배정
np.random.seed(1234)


# train/test split
test_ratio = 0.2
validation_ratio = 0.2
total_size = len(X_with_bias)

test_size = int(total_size * test_ratio)
validation_size = int(total_size * validation_ratio)
train_size = total_size - test_size - validation_size

rnd_indices = np.random.permutation(total_size)    # 150을 무작위로 섞음 

X_train = X_with_bias[rnd_indices[:train_size]]
y_train = y[rnd_indices[:train_size]]
X_valid = X_with_bias[rnd_indices[train_size:-test_size]]  # test_size 만큼 남겨둠
y_valid = y[rnd_indices[train_size:-test_size]]
X_test = X_with_bias[rnd_indices[-test_size:]]   
y_test = y[rnd_indices[-test_size:]]


# 클래스를 OneHot Vector로 바꾸기 
def to_one_hot(y):
    n_classes = y.max()+1    # 0,1,2 라 max=2 / +1 하면 classes 개수
    m = len(y)                     # 총 150개의 라벨들
    y_one_hot = np.zeros((m,n_classes))
    y_one_hot[np.arange(m),y] = 1   # index의 행중에 y값을 1로 치환
    return y_one_hot


# 라벨 전부를 onehot encoding하기
y_train_one_hot = to_one_hot(y_train)
y_valid_one_hot = to_one_hot(y_valid)
y_test_one_hot = to_one_hot(y_test)


# Softmax 함수 만들기
def softmax(logits):
    exps = np.exp(logits)
    exp_sums = np.sum(exps,axis=1,keepdims=True)   # axis=1   ->  가장 안쪽의 [ ] 안의 성분의 합 / 각 exps들의 합
    return exps/exp_sums


# 입력과 출력의 개수 정하기
n_inputs = X_train.shape[1]   # (90,3) 인데 1로 인덱싱 ==3
n_outputs = len(np.unique(y_train))  # y_train값을 중복되지 않는 값들을 출력  3


eta = 0.01
n_iteration = 5001
m = len(X_train)
epsilon = 1e-7 # ε : 입실론     nan값을 피하기 위해 logPi에 추가.

Theta = np.random.randn(n_inputs,n_outputs)

for i in range(n_iteration):
    logits = X_train.dot(Theta)
    Y_proba = softmax(logits)
    loss = -np.mean(np.sum(y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
    error = Y_proba - y_train_one_hot
    if i % 500 == 0:
        print(i,loss)
    gradients = 1/m * X_train.T.dot(error)
    Theta = Theta - eta * gradients 


# 검증 세트에 대한 정확도 확인 
logits = X_valid.dot(Theta)
y_proba = softmax(logits)
y_predict = np.argmax(y_proba, axis=1)

accuracy = np.mean(y_predict == y_valid)
accuracy